# Note
For continous training only run the "for cont'" cell

# Preparation

In [1]:
import os

MAIN_PATH = os.getcwd()
TRAINING_PATH = os.path.join(MAIN_PATH, "training")
DATA_PATH = os.path.join(MAIN_PATH, "data")
CONFIG_PATH = os.path.join(DATA_PATH, "pipeline.config")
LABEL_MAP_PATH = os.path.join(DATA_PATH, "label_map.pbtxt")
TRAIN_DATA_PATH = os.path.join(DATA_PATH, "train.record")
VAL_DATA_PATH = os.path.join(DATA_PATH, "validation.record")
TEST_DATA_PATH = os.path.join(DATA_PATH, "test.record")

print(TRAIN_DATA_PATH)

/root/train1/data/train.record


## Download the dataset

In [2]:
import requests

urls = ['https://dl.dropboxusercontent.com/s/9f5suk5oo5u7yyi/test.record',
        'https://dl.dropboxusercontent.com/s/tqvtgswib6r4f8u/validation.record',
        'https://dl.dropboxusercontent.com/s/6kgr2pwi7kgtmwb/train.record']
names = ["test.record",
        "validation.record",
        "train.record"]

for url, name in zip(urls, names):
    r = requests.get(url, allow_redirects=True)
    open(os.path.join(DATA_PATH, name), 'wb').write(r.content)

## Clone the TensorFlow Model Garden

In [3]:
# (For cont')
%cd {MAIN_PATH}
!git clone https://github.com/tensorflow/models.git


/root/train1
Cloning into 'models'...
remote: Enumerating objects: 74170, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 74170 (delta 114), reused 222 (delta 92), pack-reused 73914
Receiving objects: 100% (74170/74170), 580.17 MiB | 18.73 MiB/s, done.
Resolving deltas: 100% (52526/52526), done.


## TF Object Detection API

### Compile/install the TF Object Detection API

CD to the models/research

In [4]:
# (For cont')
%cd {MAIN_PATH}/models/research

/root/train1/models/research


### Install the protoc files

In [5]:
!apt install protobuf-compiler -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libprotobuf-dev libprotobuf-lite17 libprotobuf17 libprotoc17
The following NEW packages will be installed:
  libprotobuf-dev libprotobuf-lite17 libprotobuf17 libprotoc17
  protobuf-compiler
0 upgraded, 5 newly installed, 0 to remove and 15 not upgraded.
Need to get 2758 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libprotobuf-lite17 amd64 3.6.1.3-2ubuntu5 [132 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libprotobuf17 amd64 3.6.1.3-2ubuntu5 [798 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libprotoc17 amd64 3.6.1.3-2ubuntu5 [646 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libprotobuf-dev amd64 3.6.1.3-2ubuntu5 [1155 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/universe amd64 protobuf-compiler a

In [6]:
# (For cont')

!protoc object_detection/protos/*.proto --python_out=.

(For cont') \
\
To avoid DNN library not found problem, open the object_detection/packages/tf2/setup.py and make sure to add these lines or you can just run the next cell
```
    'tensorflow==2.8.0',
    'tf-models-official==2.8.0',
    'tensorflow_io==0.23.1',
    'keras==2.8.0',
```



In [7]:
#!pip install tensorflow==2.8.0
#!pip install tf-models-official==2.8.0
#!pip install tensorflow_io==0.23.1
#!pip install keras==2.8.0
!pip install tf-models-official
!pip install tensorflow_io
!pip install keras

     |████████████████████████████████| 2.1 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 83.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 106.7 MB/s eta 0:00:01
     |████████████████████████████████| 47.8 MB 50.3 MB/s eta 0:00:01     |███████████████▋                | 23.3 MB 50.3 MB/s eta 0:00:01     |███████████████████████         | 34.2 MB 50.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 71.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.6 MB 43.6 MB/s eta 0:00:01
     |████████████████████████████████| 92 kB 27.5 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 54.1 MB/s eta 0:00:01
     |████████████████████████████████| 41.6 MB 27.4 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 11.7 MB 88.8 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 60.9 MB/s eta 0:00:01
  Installing build dependencies 

     |█████▌                          | 5.4 MB 127.2 MB/s eta 0:00:01

     |█████████▉                      | 9.6 MB 127.2 MB/s eta 0:00:01

     |██████████████                  | 13.6 MB 127.2 MB/s eta 0:00:01

     |██████████████████▎             | 17.8 MB 127.2 MB/s eta 0:00:01

...

     |██████████████████████████▋     | 26.0 MB 127.2 MB/s eta 0:00:01

     |██████████████████████████████▋ | 29.9 MB 127.2 MB/s eta 0:00:01

     |████████████████████████████████| 31.2 MB 127.2 MB/s eta 0:00:01

     |████████████████████████████████| 31.2 MB 127.2 MB/s 
     |████████████████████████████████| 96 kB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 142 kB 66.1 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 58.7 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 56.9 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 20.7 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 59.6 MB/s eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73032 sha256=931c9b15fc3571c69d716e50f4ee7dd536bfeb2e1e4acada1588f8bbaa1ed910
  Stored in directory: /root/.cache/pip/wheels/29/da/11/144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
  Created wheel for pycocotools: filename=pycocotools-2.0.4-cp38-cp38-linux_x86_64.whl size=418791 sha256=b5b167c588230e332ba60196d4e5523950399eb8fdf210e905d5b4dcb3233910
  Stored in directory: /root/.cache/pip/wheels/dd/e2/43/3e93cd653b3346b3d702bb0509bc611189f95d60407

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 25.9 MB 2.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [8]:
# (For cont')

!cp object_detection/packages/tf2/setup.py .
!python -m pip install  .

Processing /root/train1/models/research
     |████████████████████████████████| 12.3 MB 3.8 MB/s eta 0:00:01    |█████████████████████           | 8.0 MB 3.8 MB/s eta 0:00:02
     |████████████████████████████████| 6.9 MB 42.4 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 16.6 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 5.3 MB/s  eta 0:00:01
     |████████████████████████████████| 2.6 MB 37.8 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 21.9 MB/s eta 0:00:01
     |████████████████████████████████| 546 kB 41.9 MB/s eta 0:00:01
     |████████████████████████████████| 256 kB 57.2 MB/s eta 0:00:01
     |████████████████████████████████| 26.7 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 60.8 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 60.5 MB 77.1 MB/s eta 0:00:01


  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1692783 sha256=685aeff3c76bd6dcc3524e3fef4534a34daecee1b5944393f79f434a35d75914
  Stored in directory: /tmp/pip-ephem-wheel-cache-kff44rio/wheels/63/a7/0e/470621870b3a152ee838e90ee41621c4a103254383f3be3d97
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=43992 sha256=7ff63e0912f6942a982ed6af9113e4b906fe87bf2b35970bcebce7c0ab369826
  Stored in directory: /root/.cache/pip/wheels/bb/73/e9/d273421f5723c4bf544dcf9eb097bda94421ef8d3252699f0a
  Created wheel for crcmod: filename=crcmod-1.7-cp38-cp38-linux_x86_64.whl size=35988 sha256=94232b5737235e2293e909515c5e407502da196540853b25436262ab34987254
  Stored in directory: /root/.cache/pip/wheels/ca/5a/02/f3acf982a026f3319fb3e798a8dca2d48fafee7761788562e9
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78543 sha256=78f85fb5f008bb501e2b0f361d08aa80e0f80fc1b13b1dcde42d960fa8ee37c0
  Stored in directory

### Test the TF Object Detection installation API

In [9]:
# (For cont')

!python object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.8.10: /usr/local/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-06-03 11:44:50.897643: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-03 11:44:51.654428: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22340 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:81:00.0, compute capability: 8.6
/usr/local/lib/python3.8/dist-packages/object_detection/builders/model_builder.py:1102: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(('Building experimental DeepMAC meta-arch.'
W0603 11:44:52.240099 139655455643456 model_builder.py:11

I0603 11:44:55.863711 139655455643456 efficientnet_model.py:143] round_filter input=16 output=16
I0603 11:44:55.863807 139655455643456 efficientnet_model.py:143] round_filter input=24 output=24
I0603 11:44:55.983011 139655455643456 efficientnet_model.py:143] round_filter input=24 output=24
I0603 11:44:55.983142 139655455643456 efficientnet_model.py:143] round_filter input=40 output=40
I0603 11:44:56.101857 139655455643456 efficientnet_model.py:143] round_filter input=40 output=40
I0603 11:44:56.102000 139655455643456 efficientnet_model.py:143] round_filter input=80 output=80
I0603 11:44:56.283029 139655455643456 efficientnet_model.py:143] round_filter input=80 output=80
I0603 11:44:56.283162 139655455643456 efficientnet_model.py:143] round_filter input=112 output=112
I0603 11:44:56.462074 139655455643456 efficientnet_model.py:143] round_filter input=112 output=112
I0603 11:44:56.462207 139655455643456 efficientnet_model.py:143] round_filter input=192 output=192
I0603 11:44:56.699879 13

I0603 11:44:59.396358 139655455643456 efficientnet_model.py:143] round_filter input=112 output=120
I0603 11:44:59.396511 139655455643456 efficientnet_model.py:143] round_filter input=192 output=208
I0603 11:44:59.704746 139655455643456 efficientnet_model.py:143] round_filter input=192 output=208
I0603 11:44:59.704889 139655455643456 efficientnet_model.py:143] round_filter input=320 output=352
I0603 11:44:59.823991 139655455643456 efficientnet_model.py:143] round_filter input=1280 output=1408
I0603 11:44:59.848297 139655455643456 efficientnet_model.py:453] Building model efficientnet with params ModelConfig(width_coefficient=1.1, depth_coefficient=1.2, resolution=260, dropout_rate=0.3, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_rati

I0603 11:45:04.017942 139655455643456 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet EfficientNet backbone version: efficientnet-b5
I0603 11:45:04.018087 139655455643456 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num filters: 288
I0603 11:45:04.018143 139655455643456 ssd_efficientnet_bifpn_feature_extractor.py:148] EfficientDet BiFPN num iterations: 7
I0603 11:45:04.019767 139655455643456 efficientnet_model.py:143] round_filter input=32 output=48
I0603 11:45:04.034830 139655455643456 efficientnet_model.py:143] round_filter input=32 output=48
I0603 11:45:04.034976 139655455643456 efficientnet_model.py:143] round_filter input=16 output=24
I0603 11:45:04.197316 139655455643456 efficientnet_model.py:143] round_filter input=16 output=24
I0603 11:45:04.197453 139655455643456 efficientnet_model.py:143] round_filter input=24 output=40
I0603 11:45:04.515524 139655455643456 efficientnet_model.py:143] round_filter input=24 output=40
I0603 11:45:04.515657 13

I0603 11:45:09.983742 139655455643456 efficientnet_model.py:143] round_filter input=16 output=32
I0603 11:45:09.983876 139655455643456 efficientnet_model.py:143] round_filter input=24 output=48
I0603 11:45:10.426341 139655455643456 efficientnet_model.py:143] round_filter input=24 output=48
I0603 11:45:10.426479 139655455643456 efficientnet_model.py:143] round_filter input=40 output=80
I0603 11:45:10.860674 139655455643456 efficientnet_model.py:143] round_filter input=40 output=80
I0603 11:45:10.860813 139655455643456 efficientnet_model.py:143] round_filter input=80 output=160
I0603 11:45:11.488518 139655455643456 efficientnet_model.py:143] round_filter input=80 output=160
I0603 11:45:11.488653 139655455643456 efficientnet_model.py:143] round_filter input=112 output=224
I0603 11:45:12.108342 139655455643456 efficientnet_model.py:143] round_filter input=112 output=224
I0603 11:45:12.108477 139655455643456 efficientnet_model.py:143] round_filter input=192 output=384
I0603 11:45:12.915920 

## Download pre-trained model checkpoint

In [10]:
%cd {DATA_PATH}

/root/train1/data


In [11]:
!apt install wget -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.20.3-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [12]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2022-06-03 11:45:16--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 216.58.209.16, 2a00:1450:401b:808::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|216.58.209.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  66.9MB/s    in 0.3s    

2022-06-03 11:45:17 (66.9 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-

## Modify the pipeline.config (IMPORTANT!)
change the checkpoint, label map, train, and test in data/pipeline.config to below path 

In [13]:
print("Checkpoint: " + os.path.join(DATA_PATH, "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8", "checkpoint", "ckpt-0"))
print("Label map: " + LABEL_MAP_PATH)
print("Train: " + TRAIN_DATA_PATH)
print("Test: " + TEST_DATA_PATH)


Checkpoint: /root/train1/data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0
Label map: /root/train1/data/label_map.pbtxt
Train: /root/train1/data/train.record
Test: /root/train1/data/test.record


Also change add "max_to_keep=None" as parameter in tf.train.Saver() in /research/object_detection/legacy/trainer.py \
So it'll be:\
\
saver = tf.train.Saver(max_to_keep=None, \
keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours)


## Load Tensorboard

In [14]:
# (For cont')

%load_ext tensorboard
%tensorboard --logdir {TRAINING_PATH}

# Train The Model

## Fixing incompatible libraries

In [15]:
# (For cont')
#==4.5.4.60
!pip uninstall opencv-python --y
!pip uninstall opencv-contrib-python --y
!pip uninstall opencv-python-headless --y
!pip install opencv-python==4.5.5.64 
!pip install opencv-contrib-python==4.5.5.64 
!pip install opencv-python-headless==4.5.5.64 

Found existing installation: opencv-python 4.5.5.64
Uninstalling opencv-python-4.5.5.64:
  Successfully uninstalled opencv-python-4.5.5.64
Found existing installation: opencv-python-headless 4.5.5.64
Uninstalling opencv-python-headless-4.5.5.64:
  Successfully uninstalled opencv-python-headless-4.5.5.64
  Using cached opencv_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.5 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 66.7 MB 141 kB/s eta 0:00:01     |███████████████████▎            | 40.3 MB 2.4 MB/s eta 0:00:12
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
  Using cached opencv_python_headless-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (47.8 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## Training

In [ ]:
# (For cont')

%cd {MAIN_PATH}/models/research/object_detection

!python model_main_tf2.py \
--pipeline_config_path={CONFIG_PATH} \
--model_dir={TRAINING_PATH} \
--alsologtostderr

/root/train1/models/research/object_detection
2022-06-03 11:47:18.157560: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-03 11:47:18.635153: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22342 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:81:00.0, compute capability: 8.6
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0603 11:47:18.893779 140608930690880 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0603 11:47:18.897588 140608930690880 config_util.py

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0603 11:48:21.802582 140608930690880 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0603 11:48:21.803219 140608930690880 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0603 11:48:21.804660 140608930690880 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Red

INFO:tensorflow:Step 1100 per-step time 0.681s
I0603 12:02:13.295929 140608930690880 model_lib_v2.py:705] Step 1100 per-step time 0.681s
INFO:tensorflow:{'Loss/classification_loss': 0.37353233,
 'Loss/localization_loss': 0.2503351,
 'Loss/regularization_loss': 0.15014488,
 'Loss/total_loss': 0.77401227,
 'learning_rate': 0.07999918}
I0603 12:02:13.296183 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss': 0.37353233,
 'Loss/localization_loss': 0.2503351,
 'Loss/regularization_loss': 0.15014488,
 'Loss/total_loss': 0.77401227,
 'learning_rate': 0.07999918}
INFO:tensorflow:Step 1200 per-step time 0.672s
I0603 12:03:20.526324 140608930690880 model_lib_v2.py:705] Step 1200 per-step time 0.672s
INFO:tensorflow:{'Loss/classification_loss': 0.27777284,
 'Loss/localization_loss': 0.19842972,
 'Loss/regularization_loss': 0.14971128,
 'Loss/total_loss': 0.62591386,
 'learning_rate': 0.079996705}
I0603 12:03:20.526637 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss':

INFO:tensorflow:Step 2600 per-step time 0.680s
I0603 12:19:06.716856 140608930690880 model_lib_v2.py:705] Step 2600 per-step time 0.680s
INFO:tensorflow:{'Loss/classification_loss': 0.29038015,
 'Loss/localization_loss': 0.20065112,
 'Loss/regularization_loss': 0.14521174,
 'Loss/total_loss': 0.63624305,
 'learning_rate': 0.07978972}
I0603 12:19:06.717167 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss': 0.29038015,
 'Loss/localization_loss': 0.20065112,
 'Loss/regularization_loss': 0.14521174,
 'Loss/total_loss': 0.63624305,
 'learning_rate': 0.07978972}
INFO:tensorflow:Step 2700 per-step time 0.676s
I0603 12:20:14.266096 140608930690880 model_lib_v2.py:705] Step 2700 per-step time 0.676s
INFO:tensorflow:{'Loss/classification_loss': 0.27368408,
 'Loss/localization_loss': 0.2181622,
 'Loss/regularization_loss': 0.14478938,
 'Loss/total_loss': 0.63663566,
 'learning_rate': 0.07976264}
I0603 12:20:14.266369 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss':

INFO:tensorflow:Step 4100 per-step time 0.679s
I0603 12:36:06.487922 140608930690880 model_lib_v2.py:705] Step 4100 per-step time 0.679s
INFO:tensorflow:{'Loss/classification_loss': 0.21200404,
 'Loss/localization_loss': 0.1499937,
 'Loss/regularization_loss': 0.1404546,
 'Loss/total_loss': 0.5024523,
 'learning_rate': 0.07921253}
I0603 12:36:06.488162 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss': 0.21200404,
 'Loss/localization_loss': 0.1499937,
 'Loss/regularization_loss': 0.1404546,
 'Loss/total_loss': 0.5024523,
 'learning_rate': 0.07921253}
INFO:tensorflow:Step 4200 per-step time 0.676s
I0603 12:37:14.116645 140608930690880 model_lib_v2.py:705] Step 4200 per-step time 0.676s
INFO:tensorflow:{'Loss/classification_loss': 0.2214981,
 'Loss/localization_loss': 0.1626083,
 'Loss/regularization_loss': 0.14013563,
 'Loss/total_loss': 0.52424204,
 'learning_rate': 0.07916109}
I0603 12:37:14.116959 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss': 0.2214

INFO:tensorflow:Step 5600 per-step time 0.676s
I0603 12:53:03.652342 140608930690880 model_lib_v2.py:705] Step 5600 per-step time 0.676s
INFO:tensorflow:{'Loss/classification_loss': 0.21695334,
 'Loss/localization_loss': 0.204484,
 'Loss/regularization_loss': 0.13627861,
 'Loss/total_loss': 0.55771595,
 'learning_rate': 0.07827295}
I0603 12:53:03.652588 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss': 0.21695334,
 'Loss/localization_loss': 0.204484,
 'Loss/regularization_loss': 0.13627861,
 'Loss/total_loss': 0.55771595,
 'learning_rate': 0.07827295}
INFO:tensorflow:Step 5700 per-step time 0.679s
I0603 12:54:11.512505 140608930690880 model_lib_v2.py:705] Step 5700 per-step time 0.679s
INFO:tensorflow:{'Loss/classification_loss': 0.20070107,
 'Loss/localization_loss': 0.13559203,
 'Loss/regularization_loss': 0.13600221,
 'Loss/total_loss': 0.4722953,
 'learning_rate': 0.07819763}
I0603 12:54:11.512825 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss': 0.2

INFO:tensorflow:Step 7100 per-step time 0.686s
I0603 13:10:04.283212 140608930690880 model_lib_v2.py:705] Step 7100 per-step time 0.686s
INFO:tensorflow:{'Loss/classification_loss': 0.20685662,
 'Loss/localization_loss': 0.17366773,
 'Loss/regularization_loss': 0.13217743,
 'Loss/total_loss': 0.51270175,
 'learning_rate': 0.07697967}
I0603 13:10:04.283471 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss': 0.20685662,
 'Loss/localization_loss': 0.17366773,
 'Loss/regularization_loss': 0.13217743,
 'Loss/total_loss': 0.51270175,
 'learning_rate': 0.07697967}
INFO:tensorflow:Step 7200 per-step time 0.680s
I0603 13:11:12.323184 140608930690880 model_lib_v2.py:705] Step 7200 per-step time 0.680s
INFO:tensorflow:{'Loss/classification_loss': 0.18474552,
 'Loss/localization_loss': 0.15714565,
 'Loss/regularization_loss': 0.13195391,
 'Loss/total_loss': 0.47384506,
 'learning_rate': 0.07688115}
I0603 13:11:12.323447 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss'

INFO:tensorflow:Step 8600 per-step time 0.681s
I0603 13:27:05.650289 140608930690880 model_lib_v2.py:705] Step 8600 per-step time 0.681s
INFO:tensorflow:{'Loss/classification_loss': 0.20152351,
 'Loss/localization_loss': 0.17120093,
 'Loss/regularization_loss': 0.12912562,
 'Loss/total_loss': 0.50185007,
 'learning_rate': 0.075344615}
I0603 13:27:05.650541 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss': 0.20152351,
 'Loss/localization_loss': 0.17120093,
 'Loss/regularization_loss': 0.12912562,
 'Loss/total_loss': 0.50185007,
 'learning_rate': 0.075344615}
INFO:tensorflow:Step 8700 per-step time 0.681s
I0603 13:28:13.762632 140608930690880 model_lib_v2.py:705] Step 8700 per-step time 0.681s
INFO:tensorflow:{'Loss/classification_loss': 0.15946546,
 'Loss/localization_loss': 0.11619886,
 'Loss/regularization_loss': 0.12895969,
 'Loss/total_loss': 0.40462402,
 'learning_rate': 0.07522382}
I0603 13:28:13.762903 140608930690880 model_lib_v2.py:708] {'Loss/classification_los

INFO:tensorflow:Step 10100 per-step time 0.679s
I0603 13:43:58.317075 140608930690880 model_lib_v2.py:705] Step 10100 per-step time 0.679s
INFO:tensorflow:{'Loss/classification_loss': 0.1686947,
 'Loss/localization_loss': 0.16442586,
 'Loss/regularization_loss': 0.12537529,
 'Loss/total_loss': 0.45849586,
 'learning_rate': 0.07338293}
I0603 13:43:58.317322 140608930690880 model_lib_v2.py:708] {'Loss/classification_loss': 0.1686947,
 'Loss/localization_loss': 0.16442586,
 'Loss/regularization_loss': 0.12537529,
 'Loss/total_loss': 0.45849586,
 'learning_rate': 0.07338293}


## Validation

In [ ]:
!python model_main_tf2.py \
--model_dir={TRAINING_PATH} \
--pipeline_config_path={CONFIG_PATH} \
--checkpoint_dir={TRAINING_PATH}

## Inference/Save The Model

In [ ]:
!python exporter_main_v2.py \
--trained_checkpoint_dir={TRAINING_PATH} \
--pipeline_config_path={CONFIG_PATH} \
--output_directory {DATA_PATH}/inference_graph